Spreading Out Part 4!

2D Heat Conduction

Now we want to build out an implicit solution for 2D heat conduction

Heat Equation:

$$\frac{\partial T}{\partial t} = \alpha \left( \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} \right)$$

Previous solution used Dirichlet BC on left and bottom (T(x=0) = T(y=0) = 100 and a Neumann BC with zero flux on top and edge with $q_x = q_y = 0$

$$\left( \left. \frac{\partial T}{\partial y} \right|_{y=0.1} = q_y \right) \ \ \  and \ \ \ \left( \left. \frac{\partial T}{\partial x} \right|_{x=0.1} = q_x \right)$$

See notebook for sketch of the problem.



Implicit schemes in 2D

Using the next time step to evaluate our answer:

$$\frac{T_{ij}^{n+1} - T_{ij}^n}{\Delta t} = \alpha \left( \frac{T_{i+1,j}^{n+1} - 2T_{i,j}^{n+1} + T_{i-1,j}^{n+1}}{\Delta x^2} + \frac{T_{i,j+1}^{n+1} - 2T_{i,j}^{n+1} + T_{i,j-1}^{n+1}}{\Delta y^2} \right) $$

We will place what we don't know on the left and what we know on the right:

$$ -\frac{\alpha \Delta t}{\Delta x^2}(T_{i+1,j}^{n+1} + T_{i+1,j}^{n+1}) + \left( 1 + 2\frac{\alpha \Delta t}{\Delta x^2} + 2\frac{\alpha \Delta t}{\Delta y^2} \right) T_{i,j}^{n+1} - \frac{\alpha \Delta t}{\Delta y^2}(T_{i,j-1}^{n+1} + T_{i,j+1}^{n+1}) = T_{i,j}^n$$

To make this discussion easier, let's assume that mesh spacing is same in both directions $\Delta x = \Delta y = \delta$

$$-T_{i-1,j}^{n+1} - T_{i+1,j}^{n+1} + \left( \frac{\delta^2}{\alpha \Delta t} + 4 \right) T_{i,j}^{n+1} - T_{i,j-1}^{n+1} - T_{i,j+1}^{n+1} = \frac{\delta^2}{\alpha \Delta t} T_{i,j}^{n} $$

Just like in the 1D case we note that $T_{i,j}$ appears in the equation for $T_{i-1,j}$,$T_{i+1,j}$, $T_{i,j+1}$, $T_{i,j-1}$ and we can form linear system!

BUT How do we create the matrix in this case! 

We have 2D domain of size $L_x \times L_y$ discretized in $n_x, n_y$ points. We can divide the nodes into boundary nodes and interior nodes.

The finite difference will act on the interior nodes. If we leave the boundary nodes aside, the grid will have $(n_x -2) \dot (n_y - 2)$ nodes that need to be updated each time step. This is the number of unkowns in the linear system. The matrix will will have $((n_x -2) \dot (n_y - 2))^2$ elements (most of them zero)

To construct the matrix we will iterate over the nodes in x-major order

$$i=1, \ j=1 \\
i = 2, \ j = 1 \\
i = 3, \ j = 1 \\etc..$$

#### <img src="./figures/2D_discretization.png">  Figure 2: Layout of matrix elements in 2D problem



Boundary Conditions!

Left and bottom have DBC and right and top have NBC

Bottom Boundary

$$-T_{i-1,1}^{n+1} - T_{i+1,1}^{n+1} + \left( \frac{\delta^2}{\alpha \Delta t} + 4 \right) T_{i,1}^{n+1} - T_{i,2}^{n+1} = \frac{\delta^2}{\alpha \Delta t} T_{i,1}^{n} + T_{i,0}^{n+1} $$

Left Boundary:

$$-T_{2,j}^{n+1} + \left( \frac{\delta^2}{\alpha \Delta t} + 4 \right) T_{1,j}^{n+1} - T_{1,j-1}^{n+1} - T_{1,j+1}^{n+1} = \frac{\delta^2}{\alpha \Delta t} T_{1,j}^{n} + T_{0,j}^{n+1} $$

Right Boundary:

Say the boundary condition is: $\left. \frac{\partial T}{\partial x} \right|_{x=L_x} = q_x$. Its finite difference appx is:

$$\frac{T_{n_x-1,j}^{n+1} - T_{n_x-2,j}^{n+1}}{\delta} = q_x$$

We can write $T_{n_x-1,j}^{n+1} = \delta q_x + T_{n_x-2,j}^{n+1}$ to get the finite difference equation for $i=n_x-2$:


$$-T_{n_x-3,j}^{n+1} + \left( \frac{\delta^2}{\alpha \Delta t} + 3 \right) T_{n_x-2,j}^{n+1} - T_{n_x-2,j-1}^{n+1} - T_{n_x-2,j+1}^{n+1} = \frac{\delta^2}{\alpha \Delta t} T_{n_x-2,j}^{n} + \delta q_x $$


Top Boundary:

Neumann Boundary Conditions specify the derivative normal to the boundary: $\left. \frac{\partial T}{\partial y} \right|_{y=L_y} = q_y$ The equation for $j=n_y-2$ is

$$-T_{i-1,n_y-2}^{n+1} -T_{i+1,n_y-2}^{n+1} + \left( \frac{\delta^2}{\alpha \Delta t} + 3 \right) T_{i,n_y-2}^{n+1} - T_{i,n_y-3}^{n+1} = \frac{\delta^2}{\alpha \Delta t} T_{i,n_y-2}^{n} + \delta q_y $$

